In [1]:
import sys
sys.path.insert(0, "/Users/hugofayolle/Desktop/Bourse/Bourse_test/")
from Model.codes import Codes
from Model.constants import API_KEY, ENVIRONMENT, TEST_FOLDER, SAVE_FOLDER
from Model.env import Env
from Model.file import File
from Model.functions import Function
from Model.indicator import Indicator
from Model.stock import Stock
from Model.strategy import Strategy
from functools import partial
from itertools import product
import datetime
from Model.exploration import load_stocks, get_strategy, add_indicator, add_strategy, get_stock
import itertools
import quandl
import dill as pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
quandl.ApiConfig.api_key = API_KEY # mandatory to make api calls on quandl
print(ENVIRONMENT)

Environment: DEV


# Testing test base data

In [2]:
stocks = load_stocks()

In [3]:
def sma(df, window, column = 'CLOSE'):
    return df[column].rolling(window = window).mean()

In [4]:
SMA = Function('SMA', sma)

In [5]:
Indicator('SMA80', partial(SMA.execute, window = 80))
Indicator('SMA150', partial(SMA.execute, window = 150))

Indicator : SMA150 (DEV)

In [6]:
add_indicator('SMA80')
add_indicator('SMA150')

In [7]:
stocks = load_stocks()

In [8]:
def buy(stock, row):
    import numpy as np
    date = None
    price = None
    status = None
    if np.isnan(row.SMA80) or np.isnan(row.SMA150):
        state = True
    else:
        state = row.SMA80 > row.SMA150
        if state == True:
            pos = stock.data.index.get_loc(row.Index)
            if pos + 1 == len(stock.data):
                price = None
                status = 'awaiting buy'
                date = row.Index
            else:
                next_row = stock.data.iloc[pos + 1]
                price = next_row.OPEN
                status = 'pending'
                date = next_row.name
    return {'state': state, 'date': date, 'price': price, 'status': status}

def sell(stock, row, trade):
    date = None
    price = None
    status = None
    state = row.SMA80 < row.SMA150
    if state == True:
        pos = stock.data.index.get_loc(row.Index)
        if pos + 1 == len(stock.data):
            price = None
            status = 'awaiting sell'
            date = row.Index
        else:
            next_row = stock.data.iloc[pos + 1]
            price = next_row.OPEN
            status = 'closed'
            date = next_row.name
    return {'state': state, 'date': date, 'price': price, 'status': status}

In [12]:
Strategy('SMA80_SMA150', buy, sell)

Strategy : SMA80_SMA150 (DEV)

In [14]:
add_strategy('SMA80_SMA150')

---

# Exploration

In [34]:
from crontab import CronTab

In [35]:
cron = CronTab(user='hugofayolle')  

In [36]:
job = cron.new(command='python3 /Users/hugofayolle/Desktop/Bourse/Bourse_test/Script/dev.py') 

In [37]:
job.minute.every(1)

In [38]:
cron.write()

In [33]:
job.is_enabled()

True

In [32]:
job.is_valid()

True